In [1]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
import time
import numpy as np
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceBgeEmbeddings


load_dotenv()
## load the Groq API key
groq_api_key=os.environ['GROQ_API_KEY']

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Load pdf files in the local directory
def load_and_split_text(pdf_path):
    loader = PyPDFDirectoryLoader(pdf_path)

    # docs_before_split = loader.load()


    # text_splitter = RecursiveCharacterTextSplitter(
    #     chunk_size = 512,
    #     chunk_overlap  = 100,
    # )
    # docs_after_split = text_splitter.split_documents(docs_before_split)

    # return docs_after_split
    doc_pages = loader.load()
    return doc_pages
    

In [3]:
import re

def extract_substring_index(text, start_marker, end_marker):
    start_index = text.index(start_marker) + len(start_marker)
    end_index = text.index(end_marker, start_index)
    return text[start_index:end_index]

In [4]:

def get_metadata(text):
    AMBITO ='Ámbito Geográfico'
    INFORMACION='Información Detallada'

    document_tags = ['Referencia','Organismo','Sector','Subsector',
                    AMBITO,'Tipo','Destinatarios','Plazo de solicitud']

    tagIndex = 0
    metadata = {}
    metadataInText=""
    while tagIndex < len(document_tags)-1:
        start = document_tags[tagIndex]
        end = document_tags[tagIndex+1]
        if(start=='Ámbito Geográfico'):
            metadata[start]=extract_substring_index(text,start,end).replace(AMBITO,'').replace(INFORMACION,'').strip()
        else:
            metadata[start]=extract_substring_index(text,start,end).strip()
        metadataInText = metadataInText+", "+start+" es "+metadata[start]
        tagIndex+=1
            
        
    return [ metadata, metadataInText ]

In [5]:
from pathlib import Path
from urllib.parse import urlparse

import requests
def download_file(url,output_path,filename):
    response = requests.get(url)
    if response.status_code == 200:        
        with open(output_path+"/"+filename, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded {filename}")
    else:
        print(f"Failed to download {url}")

In [6]:

from uuid import uuid4


def download_linked_files(page, output_path):
    urls=[]
    if "/Annots" in page:
        for annot in page["/Annots"]:
            annotObj = annot.get_object()
            if("/A" in annotObj):
                uri = annotObj.get("/A").get("/URI")
                if uri is not None:
                    print("[+] URL Found:", uri)
                    urls.append(uri)
    
    if(not os.path.exists(output_path)):
        os.makedirs(output_path)
    for url in urls:
        download_file(url, output_path, str(uuid4())+".pdf")             

In [7]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="jaimevera1107/all-MiniLM-L6-v2-similarity-es",  # alternatively use "sentence-transformers/all-MiniLM-l6-v2" for a light and faster experience.
    model_kwargs={'device':'cpu'}, 
    encode_kwargs={'normalize_embeddings': False}
)

/home/dordorica/QA_PDF_Subvenciones_a_empresas/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import PyPDF2
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import chromadb
from chromadbx import UUIDGenerator
import os
from urllib.parse import urlparse

# Cargar modelo de embedding
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModelForSequenceClassification.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Crear una colección en ChromaDB
client = chromadb.PersistentClient('./db_subvenciones')
client.delete_collection("ayudas")
collection = client.create_collection("ayudas")
pathToMetadata = './ayudas/metadatos'
pathToText = './ayudas/texto'
# Función para procesar un PDF
def process_pdf(pdf_path):

    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        limit = 1
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text().replace("\n"," ")
            if limit==10:
                break
            if (text.find("Ayudas e incentivos (detalle)") > -1):
                a = urlparse(pdf_path)
                output_dir = pathToText+"/"+os.path.basename(a.path)+"/"+"Page_"+str(page_num)
                
                #Get metadata from page
                metadata = get_metadata(text)
                page_metadata = metadata[0]
                page_metadataInText = metadata[1]
                download_linked_files(page, output_dir)
    
                splitted_text = load_and_split_text(output_dir)
                
                if(len(splitted_text) > 0):
                    embeddings=[]
                    docs=[]

                    for text in splitted_text:
                        cleanstr=text.page_content.replace("\n","")
                        #Add the metadata in text format to associate it with every chung, since I
                        #consider it important search criteria
                        completestr = "("+page_metadataInText+")."+cleanstr
                        docs.append(completestr)                        
                        embeddings.append(np.array(huggingface_embeddings.embed_query(completestr)))
                    # Agregar a ChromaDB
                    collection.add(
                        ids=UUIDGenerator(len(docs)),
                        documents=docs,
                        embeddings=embeddings,
                        metadatas=[page_metadata]*len(docs)
                    )
                    
                    limit+=1

# Procesar todos los PDFs en una carpeta
import os
for file in os.listdir(pathToMetadata):
    if file.endswith(".pdf"):
        process_pdf(os.path.join(pathToMetadata, file))



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[+] URL Found: https://www.cofides.es/financiacion/internacionalizacion/pyme-invierte
Failed to download https://www.cofides.es/financiacion/internacionalizacion/pyme-invierte
[+] URL Found: https://wapis.ipyme.org/servicioayudas/ayudas/detalle?id=71572&fichero=
[+] URL Found: https://wapis.ipyme.org/servicioayudas/ayudas/detalle?id=71572&fichero=
[+] URL Found: https://wapis.ipyme.org/servicioayudas/ayudas/detalle?id=71572&fichero=
Downloaded 4a736b73-e037-453f-bef4-1fd079336d67.pdf
Downloaded 4e40fc09-af2b-411d-acf7-7ecd8d8ca504.pdf
Downloaded b4e4b91b-712e-4a22-b847-708ed3b546bd.pdf
[+] URL Found: https://wapis.ipyme.org/servicioayudas/ayudas/detalle?id=100120&fichero=
[+] URL Found: https://wapis.ipyme.org/servicioayudas/ayudas/detalle?id=100120&fichero=
Downloaded b56eb950-5bfb-410a-8f3c-6db4c09571ba.pdf
Downloaded 66202b23-1a86-4bc4-a73a-4fcbff873fde.pdf
[+] URL Found: https://wapis.ipyme.org/servicioayudas/ayudas/detalle?id=108259&fichero=
Downloaded dada8552-8718-4bda-b0c1-776c

In [9]:
query = """   71572   """  
         # Sample question, change to other questions you are interested in.
# Ejemplo de búsqueda

results = collection.query(
    query_embeddings = np.array(huggingface_embeddings.embed_query(query)),
    #query_texts = [query],
    n_results=5,
    )
print(results)
     

{'ids': [['d49ae3e1-3ef3-4213-a47c-7322dd9c648a', '764e94f3-ecae-4fb4-bbfe-4fa81654494e', 'ab5cb80b-275d-4e83-8c59-4dcd010edc5a', 'ff5a04f4-212e-4440-9c0e-38b1ebd12d03', '5bae78ea-85aa-4b35-8c7c-dacc1f3112c7']], 'embeddings': None, 'documents': [['(, Referencia es 71572 Título Se convoca, en régimen de concurrencia competitiva, la concesión de subvenciones  públicas destinadas al incremento de la competitividad del comercio de proximidad en  el territorio de Ceuta, anualidad 2019, Organismo es Consejería de Economía, Hacienda, Administraciones Públicas y Empleo, Sector es Comercio, Subsector es , Ámbito Geográfico es Ceuta, Tipo es Subvención, Destinatarios es Organizaciones interprofesionales, con personalidad jurídica propia y corporaciones  de derecho público).Boletín Oficial de la Ciudad de Ceuta  -  Plaza de África S/N                                            2.094 693.- TituloES: Convocatoria de subvenciones destinadas al incremento de la competitividad del comercio de proximid

In [10]:
import chromadb

# Use similarity searching algorithm and return 3 most relevant documents.

db = Chroma(client=client, collection_name="ayudas",embedding_function=huggingface_embeddings)

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

/tmp/ipykernel_236335/1269315768.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(client=client, collection_name="ayudas",embedding_function=huggingface_embeddings)


In [11]:
# Remote huggingface execution
# from langchain_community.llms import HuggingFaceHub

# hf = HuggingFaceHub(
#     repo_id="stabilityai/stablelm-2-1_6b",
#     model_kwargs={"temperature":0.1, "max_length":500})

# query = """What were the trends in median household income across different states in the United States between 2021 and 2022."""  # Sample question, change to other questions you are interested in.
# hf.invoke(query)

In [29]:
llm=ChatGroq(groq_api_key=groq_api_key,
model_name="gemma2-9b-it")

In [30]:
from langchain.chains import ReduceDocumentsChain

qa_template = """Eres un asistente para responder a preguntas. "
    "Debes reproducir exactamente el fragmento de texto donde viene la respuesta"
    "Ordena todas las respuestas que encuentres en diferentes líneas,"
    "Si algún documento no contiene la respuesta, ignóralo. Si no conoces la respuesta, simplemente di "
    "que no lo sabes."
    "\n\n"
    "{context}"

Pregunta: {question}
Respuesta:"""

prompt = PromptTemplate(template=qa_template,
                            input_variables=['context','question'])
combine_custom_prompt='''
Responde con todas las respuestas que encuentres en diferentes documentos.

Text:`{context}`
'''


combine_prompt_template = PromptTemplate(
    template=combine_custom_prompt, 
    input_variables=['context']
)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type="map_reduce",return_source_documents=True,
 chain_type_kwargs= {
        "verbose": False,
        "question_prompt": prompt,
        "combine_prompt": combine_prompt_template,
        "combine_document_variable_name": "context"})

question = "Qué ayudas da el Ayuntamiento de Basauri" 

result=qa_chain.invoke(question)
print(result['result'])

## Respuestas a preguntas sobre las ayudas al comercio en Basauri 2024:

**Destinatarios:**

* Personas físicas o jurídicas o comunidades de bienes que acometan la reforma o modernización de establecimientos comerciales.

**Ámbito geográfico:**

* Basauri

**Organismo:**

* Ayuntamiento de Basauri

**Administración:**

* Administración Local

**Plazo de solicitud:**

* Hasta el 31 de diciembre de 2024

**Tipo de ayuda:**

* Subvención

**Importe total:**

* 90.000,00€

**Cuantía máxima que se subvenciona:**

* 40% de los gastos y/o inversiones, con un límite de 1.000 euros por solicitud.

**Referencias legales:**

* Decreto de Alcaldía 360/24. Boletín Oficial de Bizkaia número 33 de 15 de febrero de 2024. (Convocatoria)
* Decreto de Alcaldía 725/24. Boletín Oficial de Bizkaia número 48 de 7 de marzo de 2024. (Extracto)

**Documentación necesaria para la solicitud:**

* Memoria descriptiva de las inversiones, reformas o adquisiciones realizadas.
* Declaración jurada relativa al cumplimi

In [ ]:
relevant_docs = result['source_documents']
print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')
print("*" * 100)
for i, doc in enumerate(relevant_docs):
    print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}, Page: {doc.metadata['page']}\nContent: {doc.page_content}")
    print("-"*100)
    print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')